In [1]:
import numpy as np

import tensorflow as tf

import tensorflow_datasets as tfds

In [79]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [80]:
mnist_train=mnist_dataset['train']
mnist_test=mnist_dataset['test']



num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)



num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image,label):
    image=tf.cast(image,tf.float32)
    image /= 255.

    return image,label


scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)



In [81]:
BUFFER_SIZE=10000
shuffled_train_and_validation_data=scaled_train_and_validation_data.shuffle(BUFFER_SIZE)


validation_data=shuffled_train_and_validation_data.take(num_validation_samples)


BATCH_SIZE = 100
train_data=shuffled_train_and_validation_data.skip(num_validation_samples)


train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data=test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))



### Model

In [90]:
input_size=784
output_size=10
hidden_layer_size=200

model=tf.keras.Sequential([ tf.keras.layers.Flatten(input_shape=(28,28,1)),
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                           tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                           tf.keras.layers.Dense(output_size,activation='softmax')    
                            ])

In [91]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])


In [92]:
NUM_EPOCHS=5
model.fit(train_data,epochs=NUM_EPOCHS,validation_data=(validation_inputs, validation_targets),validation_steps=10,verbose=2)


Epoch 1/5
540/540 - 3s - loss: 0.2641 - accuracy: 0.9227 - val_loss: 0.1206 - val_accuracy: 0.9643
Epoch 2/5
540/540 - 3s - loss: 0.1019 - accuracy: 0.9685 - val_loss: 0.0870 - val_accuracy: 0.9753
Epoch 3/5
540/540 - 2s - loss: 0.0700 - accuracy: 0.9774 - val_loss: 0.0758 - val_accuracy: 0.9770
Epoch 4/5
540/540 - 3s - loss: 0.0523 - accuracy: 0.9832 - val_loss: 0.0529 - val_accuracy: 0.9843
Epoch 5/5
540/540 - 2s - loss: 0.0403 - accuracy: 0.9867 - val_loss: 0.0547 - val_accuracy: 0.9850


In [93]:
test_loss, test_accuracy = model.evaluate(test_data)


      1/Unknown - 1s 1s/step - loss: 0.0871 - accuracy: 0.9733

In [94]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.09. Test accuracy: 97.33%
